In [12]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [10]:
load_dotenv(override=True)
MAIL_KEY = os.getenv("MAIL_KEY")

In [54]:
@function_tool
def send_email( Subject:str, content:str )->Dict[str,str]:
    sender_email = "agasthybot@gmail.com"
    receiver_email = "agasthynathgs@gmail.com"
    app_password   = MAIL_KEY # Replace this

    # Create message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = Subject
    body = content
    message.attach(MIMEText(body, "plain"))

    # Connect to Gmail SMTP
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(sender_email, app_password)
    server.sendmail(sender_email, receiver_email, message.as_string())
    server.quit()

    return {"status" : "success"}


In [53]:
response = send_email("test","testing mail 2")
print(response["status"])

success


## Step 1: Agent workflow

In [82]:
instructions1 = "You are a sales agent working for BrokeAI, \
a company that provides a Agentic AI solution for customer support and customer complience. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for BrokeAI, \
a company that provides a Agentic AI solution for customer support and customer complience. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for BrokeAI, \
a company that provides a Agentic AI solution for customer support and customer complience. \
You write concise, to the point cold emails."

In [83]:
Agent1 = Agent(name = "proffetional agent",instructions=instructions1,model="gpt-4o-mini")
Agent2 = Agent(name="engaging agent",instructions=instructions2,model="gpt-4o-mini")
Agent3 = Agent(name="busy agent",instructions=instructions3,model="gpt-4o-mini")

In [65]:
#testing the runner
result =  Runner.run_streamed(Agent1, "Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Transform Your Customer Support with Agentic AI Solutions

Hi [Recipient's Name],

I hope this message finds you well.

My name is [Your Name], and I represent BrokeAI, where we specialize in advanced AI solutions designed to enhance customer support and compliance processes. 

In today’s fast-paced business environment, delivering exceptional customer service is more crucial than ever. Our Agentic AI technology not only streamlines customer interactions but also ensures compliance with industry standards, allowing your team to focus on what they do best.

Here are a few ways our solution can benefit your organization:

- **24/7 Customer Support:** Provide instant responses to queries, reducing wait times and improving customer satisfaction.
- **Automated Compliance Checks:** Ensure adherence to regulations automatically, minimizing risks and potential liabilities.
- **Scalable Solutions:** Easily adapt to business growth without the need for extensive human resources.

I woul

In [66]:
# setting an agent as sales manager who deside the best mail
sales_manager = Agent(name="sales_manager",model="gpt-4o-mini", instructions= "You pick the best cold sales email from the given options. \
                                                                                    Imagine you are a sales manager and pick the one you are most likely to send to customer's. \
                                                                                    Do not give an explanation; reply with the selected email only.")


In [67]:
# executing the sales manager agent to find the best mail
message = "Write a cold sales email"
with trace("test"):
    results = await asyncio.gather(
        Runner.run(Agent1, message),
        Runner.run(Agent2, message),
        Runner.run(Agent3, message))
    predictions = [result.final_output for result in results]
    emails      = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(predictions)
    best_email  = await Runner.run(sales_manager, emails)

    print(f"Best sales email:\n{best_email.final_output}")
    

Best sales email:
Subject: Enhance Your Customer Support with Agentic AI

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent BrokeAI, where we specialize in streamlining customer support with our cutting-edge Agentic AI solution.

In today’s fast-paced environment, businesses face the challenge of meeting rising customer expectations while maintaining operational efficiency. Our Agentic AI not only enhances response times but also ensures compliance with industry regulations, allowing your team to focus on what truly matters — building lasting relationships with your customers.

By integrating our solution, you can expect:

- **24/7 Availability**: Provide immediate assistance, even outside business hours.
- **Personalized Interactions**: Utilize data analytics to engage with customers on a more personal level.
- **Compliance Assurance**: Stay ahead of regulatory requirements effortlessly.

I would love the opportunity to discuss how B

To check out the trace:

https://platform.openai.com/traces

## Steps 2 and 3: Tools and Agent interactions

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [85]:
#setting up agents as tools
description = "Write a cold sales email"
tool1 = Agent1.as_tool(tool_name="sales_Agent1",tool_description=description)
tool2 = Agent2.as_tool(tool_name="sales_Agent2",tool_description=description)
tool3 = Agent3.as_tool(tool_name="sales_Agent3",tool_description=description)

tools = [tool1,tool2,tool3,send_email]
tools

[FunctionTool(name='sales_Agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_Agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115fe89a0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_Agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_Agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115fe9ee0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_Agent3', description='Write 

## And now it's time for our Sales Manager - our planning agent

In [87]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales Agents tools.
 
Follow these steps carefully:

1. Generate Drafts: Use all three sales Agents tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective for customer.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the Agents tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
- Do not pass any instruction to sales Agents, they are already given their instructions.
- You must strictly follow the rules.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)



https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [95]:

subject_instructions = "You can write a subject for a cold sales email. \
                        You are given a message and you need to write a subject for an email that is likely to get a response."

spell_check_instructions = "you will recieve cold email from the emailer agent check any spellings or grammer are gone wrong , \
                            if its wrong try to pass where the changes needed to be done else dont pass anything."

subject_writer = Agent(name="Email_subject_writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")


spell_checker = Agent(name="Email_spell_checker", instructions=spell_check_instructions, model="gpt-4o-mini")
spell_check_tool = spell_checker.as_tool(tool_name="spell_checker", tool_description="Check the spelling for a cold sales email")

emailer_tools = [subject_tool, spell_check_tool ,send_email]
emailer_tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1162313a0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='spell_checker', description='Check the spelling for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'spell_checker_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x116231d00>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(nam

In [102]:
emailer_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, \
then use spell checker tool for check the grammer and spelling of the cold mail, \
Finally, you use the send_email tool to send the email with the subject and update the mailby changing the suggetions from spell checker .\
    Crucial Rules:\
        1. must use subject_writter tool for getting the subject, do not need to pass any instructions, pass the cold mail as a message so he can get a good subject for the mail.\
        2. must use spell_check tool to get suggetions to change any grammer or spell checks in the cold mail and change accordingly the mail, do not send any instructions to this tool\
        3. use send mail tool for sending the mail to customer , strictly follow the rules"


emailer_agent = Agent(
    name="Email Manager",
    instructions=emailer_instructions,
    tools=emailer_tools,
    model="gpt-4o-mini",
    handoff_description="add subject and recorrect the mail and send it")


### Now we have 3 tools and 1 handoff

In [103]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_Agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_Agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115fe89a0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_Agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_Agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x115fe9ee0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_Agent3', description='Write a 

In [104]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at BrokeAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try. after Finalizing the one mail dont run the sales agents anymore.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Emailer — never more than one.
- Do not pass any instruction to sales Agents and Emailer, they are already given their instructions.
- You must strictly follow the rules.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)



https://platform.openai.com/traces
